## Features extraction

In [1]:
import numpy as np
import pandas as pd
import torch
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from matplotlib.colors import ListedColormap
import plotly.express as px
import plotly.graph_objs as go
import subprocess
import os
from umap.umap_ import UMAP


c:\Users\iplov\miniconda3\envs\aml_ego\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
LABELS = [
    "get, take",
    "place, place-on, place-back, put, put-back",
    "open",
    "close",
    "clean, wipe-off, wipe, wash, rinse",
    "cut, chop, slice",
    "mix, mix-around, stir, whisk",
    "pour"
]

In [ ]:
if False:   

    # Assicurati di essere nella directory corretta
    os.chdir("C:/Users/iplov/Desktop/Advanced ML/Progetto/github andrew/Egocentric-Vision")
    # Comando da eseguire
    command = [
        'python', 'save_feat.py',
        'config=configs/I3D_save_feat.yaml',
        'dataset.shift=D1-D1',
        'dataset.RGB.data_path=D:/ek_data/frames',
        'save.dense_sampling.RGB=true',
        'save.num_frames_per_clip.RGB=5',
        'name=saved_feat_I3D_5_Dense'
    ]

    # Esegui il comando
    result = subprocess.run(command, capture_output=True, text=True)

    # Stampa l'output e l'errore
    print('Output:', result.stdout)
    #print('Error:', result.stderr)

    # Verifica il codice di uscita
    if result.returncode != 0:
        print(f'Command failed with return code {result.returncode}')

In [ ]:
# Assicurati di essere nella directory corretta
os.chdir("C:/Users/iplov/Desktop/Advanced ML/Progetto/github andrew/Egocentric-Vision")
num_frames_list = [5,10,25]
#DENSE
if False:
    for n in num_frames_list:
        # Comando da eseguire
        command = [
            'python', 'save_feat.py',
            'config=configs/I3D_save_feat.yaml',
            'dataset.shift=D1-D1',
            'dataset.RGB.data_path=D:/ek_data/frames',
            'save.dense_sampling.RGB=true',
            f'save.num_frames_per_clip.RGB={n}',
            f'name=saved_feat_I3D_{n}_dense'
        ]

        # Esegui il comando
        result = subprocess.run(command, capture_output=True, text=True)

        # Stampa l'output e l'errore
        print('Output:', result.stdout)
        #print('Error:', result.stderr)

        # Verifica il codice di uscita
        if result.returncode != 0:
            print(f'Command failed with return code {result.returncode}')

    #UNIFORM
    for n in num_frames_list:
        print(f"UNIFORM n = {n}")
        # Comando da eseguire
        command = [
            'python', 'save_feat.py',
            'config=configs/I3D_save_feat.yaml',
            'dataset.shift=D1-D1',
            'dataset.RGB.data_path=D:/ek_data/frames',
            'save.dense_sampling.RGB=false',
            f'save.num_frames_per_clip.RGB={n}',
            f'name=saved_feat_I3D_{n}_uniform'
        ]

        # Esegui il comando
        result = subprocess.run(command, capture_output=True, text=True)

        # Stampa l'output e l'errore
        print('Output:', result.stdout)
        #print('Error:', result.stderr)

        # Verifica il codice di uscita
        if result.returncode != 0:
            print(f'Command failed with return code {result.returncode}')

### Extract train feature

In [ ]:
# Assicurati di essere nella directory corretta
os.chdir("/Users/andreavannozzi/GithubProjects/Egocentric-Vision")
num_frames_list = [5,10,25]
#DENSE
if True:
    for n in num_frames_list:



        print(f"UNIFORM n = {n}")
        # Comando da eseguire
        command = [
            'python', 'save_feat.py',
            'config=configs/I3D_save_feat.yaml',
            'dataset.shift=D1-D1',
            'dataset.RGB.data_path=./ek_data/frames',
            'save.dense_sampling.RGB=false',
            f'save.num_frames_per_clip.RGB={n}',
            f'name=saved_feat_I3D_{n}_uniform',
            'split=train'
        ]

        # Esegui il comando
        result = subprocess.run(command, capture_output=True, text=True)

        # Stampa l'output e l'errore
        print('Output:', result.stdout)
        print('Error:', result.stderr)

        # Verifica il codice di uscita
        if result.returncode != 0:
            print(f'Command failed with return code {result.returncode}')

        

## Read extracted feature

In [ ]:
features = pd.read_pickle("./saved_features/saved_feat_I3D_10_dense_D1_test.pkl")
#print(features['features'][0])
print(len(features['features']))
#print(features['features'][0]['features_RGB'])
#print(features['features'][0]['label'])
print(features['features'][0].keys())
list_of_features = [np.mean(np.array(feature['features_RGB']),axis=0) for feature in features["features"]]
clips_feature = list_of_features
labels = [feature['label'] for feature in features["features"]]
labels_extended = [label for label in labels for _ in range(5)]
print(labels_extended[0:20])
list_of_features = [np.array(f) for feature in features["features"] for f in feature['features_RGB']]

In [ ]:
features = pd.read_pickle("./train_val/D1_test.pkl")
#print(features['features'][0])
print(features.keys())
print(len(features))
#print(features['features'][0]['features_RGB'])
#print(features['features'][0]['label'])

print(len(set(features['verb'])))

# K-Means

In [ ]:
clips_feature_tensor = np.array(clips_feature)

print(clips_feature_tensor.shape)

## plot using TSNE 2D

In [ ]:
#plot k-means labels
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(clips_feature_tensor)
cmap = ListedColormap(['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black', 'olive'])

y_kmeans = kmeans.predict(clips_feature_tensor)

tsne = TSNE(n_components=2, random_state=42)
# Addestrare il modello e trasformare i dati
X = tsne.fit_transform(clips_feature_tensor)
# scatter plot of X values
plt.figure(1)
plt.scatter(X[:, 0], X[:, 1], c=y_kmeans,cmap=cmap, s=50)

plt.show()

#plot real labels
cmap = ListedColormap(['gray', 'orange', 'black', 'red', 'green', 'olive', 'purple', 'blue'])
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cmap(i),  markersize=10, label=desc) 
                for i, desc in enumerate(LABELS)]
plt.figure(2)

# scatter plot of X values
plt.scatter(X[:, 0], X[:, 1], c=labels, cmap=cmap, s=50)
# Add the legend
plt.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1))


plt.show()


## plot using TSNE 3D

In [ ]:
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(clips_feature_tensor)
cmap = ['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black', 'olive']
kmeans_color_map = {f"{i}": color for i, color in enumerate(cmap)}

y_kmeans = kmeans.predict(clips_feature_tensor)

tsne = TSNE(n_components=3, random_state=42)
# Addestrare il modello e trasformare i dati
X = tsne.fit_transform(clips_feature_tensor)
# scatter plot of X values
df = pd.DataFrame(X, columns=['x', 'y', 'z'])
df['kmeans'] = y_kmeans.astype(str)
df['labels'] = np.array(labels).astype(str)


#plot label k_means
fig = px.scatter_3d(df, x='x', y='y', z='z', color="kmeans", color_discrete_map=kmeans_color_map, title="plot using k_means label")
fig.update_traces(marker_size=4)
fig.show()


cmap = ['blue', 'orange', 'black', 'red', 'green', 'olive', 'purple', 'gray']
#plot real label
fig = go.Figure()
for i, label in enumerate(LABELS):
    class_data = df[df['labels'] == str(i)]
    fig.add_trace(go.Scatter3d(
        x=class_data['x'], y=class_data['y'], z=class_data['z'],
        mode='markers',
        marker=dict(size=10, color=cmap[i]),
        name=label
    ))
fig.update_traces(marker_size=4)
fig.show()

## UMAP 2D


In [ ]:
umap_2d = UMAP(n_components=2, init='random', random_state=0)
proj_2d = umap_2d.fit_transform(clips_feature_tensor)


#plot k-means labels
kmeans = KMeans(n_clusters=8, random_state=0, n_init="auto").fit(clips_feature_tensor)
cmap = ListedColormap(['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black', 'olive'])

y_kmeans = kmeans.predict(clips_feature_tensor)

# scatter plot of X values
plt.figure(1)
plt.scatter(proj_2d[:, 0], proj_2d[:, 1], c=y_kmeans,cmap=cmap, s=50)

plt.show()

#plot real labels
cmap = ListedColormap(['gray', 'orange', 'black', 'red', 'green', 'olive', 'purple', 'blue'])
legend_handles = [plt.Line2D([0], [0], marker='o', color='w', markerfacecolor=cmap(i),  markersize=10, label=desc) 
                for i, desc in enumerate(LABELS)]
plt.figure(2)

# scatter plot of X values
plt.scatter(proj_2d[:, 0], proj_2d[:, 1], c=labels, cmap=cmap, s=50)
# Add the legend
plt.legend(handles=legend_handles, loc='upper left', bbox_to_anchor=(1, 1))


plt.show()


## UMAP 3D

In [ ]:
cmap = ['blue', 'orange', 'green', 'red', 'purple', 'gray', 'black', 'olive']
color_map = {f"{i}": color for i, color in enumerate(cmap)}

umap_3d = UMAP(n_components=3, init='random', random_state=0)
proj_3d = umap_3d.fit_transform(clips_feature_tensor)

df3d = pd.DataFrame(proj_3d, columns=['x', 'y', 'z'])
df3d['kmeans'] = y_kmeans.astype(str)
df3d['labels'] = np.array(labels).astype(str)

#plot kmeans
fig_3d = px.scatter_3d(
    df3d, x='x', y='y', z='z',
    color='kmeans', color_discrete_map=color_map
)
fig_3d.update_traces(marker_size=5)
fig_3d.show()


cmap = ['blue', 'orange', 'black', 'red', 'green', 'gray', 'purple', 'olive']
kmeans_color_map = {f"{i}": color for i, color in enumerate(cmap)}
custom_legend = {f"{i}": label_name for i, label_name in enumerate(LABELS)}
#plot real label
fig = go.Figure()
for i, label in enumerate(LABELS):
    class_data = df3d[df3d['labels'] == str(i)]
    fig.add_trace(go.Scatter3d(
        x=class_data['x'], y=class_data['y'], z=class_data['z'],
        mode='markers',
        marker=dict(size=10, color=cmap[i]),
        name=label
    ))
fig.update_traces(marker_size=5)
fig.show()

# read actionet

In [2]:
features1 = pd.read_pickle("./action-net/ActionNet_train.pkl")
features2 = pd.read_pickle("./action-net/ActionNet_test.pkl")
#print(features['features'][0])
print(features1.keys())
print(len(features1))
#print(features['features'][0]['features_RGB'])
#print(features['features'][0]['label'])
l = list(zip(features1['index'][:],features1['file'][:],features1['description'][:],features1['labels'][:]))
for el in l:
    pass
    #print(el)
#print(len(set(features['description'])))
#print(sorted(set(features['description'])))

s04 = features2[features2['file'] == 'S00_2.pkl']
print(len(set(s04['index'])))
print(sorted(np.array(s04['index']) - 1))
print(len(set(s04['index'])))
print(len(list(s04['index'])))
print(f"file train: {sorted(set(features1['file']))}")
print(f"file test: {sorted(set(features2['file']))}")
s04
#['Clean a pan with a sponge', 'Clean a pan with a towel', 'Clean a plate with a sponge', 'Clean a plate with a towel', 'Clear cutting board', 'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Get items from refrigerator/cabinets/drawers', 'Get/replace items from refrigerator/cabinets/drawers', 'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Open a jar of almond butter', 'Open/close a jar of almond butter', 'Peel a cucumber', 'Peel a potato', 'Pour water from a pitcher into a glass', 'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Slice a cucumber', 'Slice a potato', 'Slice bread', 'Spread almond butter on a bread slice', 'Spread jelly on a bread slice', 'Stack on table: 3 each large/small plates, bowls', 'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils']
#['Clean a pan with a towel', 'Clean a plate with a sponge', 'Clean a plate with a towel', 'Clear cutting board', 'Get items from cabinets: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Get items from refrigerator/cabinets/drawers', 'Get/replace items from refrigerator/cabinets/drawers', 'Load dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Open/close a jar of almond butter', 'Peel a cucumber', 'Peel a potato', 'Pour water from a pitcher into a glass', 'Set table: 3 each large/small plates, bowls, mugs, glasses, sets of utensils', 'Slice a cucumber', 'Slice a potato', 'Slice bread', 'Spread almond butter on a bread slice', 'Spread jelly on a bread slice', 'Stack on table: 3 each large/small plates, bowls', 'Unload dishwasher: 3 each large/small plates, bowls, mugs, glasses, sets of utensils']

#robe simili "open a jar of almond butter" == "open/close a jar of almond butter" &&&
#            "Get items from refrigerator/cabinets/drawers" == "Get/replace items from refrigerator/cabinets/drawers"

Index(['index', 'file', 'description', 'labels'], dtype='object')
527
2
[39, 51]
2
2
file train: ['S00_2.pkl', 'S01_1.pkl', 'S02_2.pkl', 'S02_3.pkl', 'S02_4.pkl', 'S03_1.pkl', 'S03_2.pkl', 'S04_1.pkl', 'S05_2.pkl', 'S06_1.pkl', 'S06_2.pkl', 'S07_1.pkl', 'S08_1.pkl', 'S09_2.pkl']
file test: ['S00_2.pkl', 'S02_2.pkl', 'S02_3.pkl', 'S03_1.pkl', 'S03_2.pkl', 'S04_1.pkl', 'S05_2.pkl', 'S06_1.pkl', 'S06_2.pkl', 'S07_1.pkl', 'S08_1.pkl', 'S09_2.pkl']


,index,file,description,labels
40,40,S00_2.pkl,Clean a plate with a towel,Clean
52,52,S00_2.pkl,"Unload dishwasher: 3 each large/small plates, ...",Unload


In [7]:
import h5py
import numpy as np
import pandas as pd
# Specifica il percorso del file HDF5
file_path = r"D:/ActionNetDataset/2022-06-07_18-11-37_streamLog_actionNet-wearables_S04_1.hdf5"

# Apri il file HDF5 in modalità lettura
with h5py.File(file_path, 'r') as hdf:
    # Stampa il contenuto del file HDF5
    print("Keys: %s" % hdf.keys())

    emg_left_data = hdf['myo-left']['emg']['data'][:]
    emg_left_time = hdf['myo-left']['emg']['time_s'][:]
    
    emg_right_data = hdf['myo-right']['emg']['data'][:]
    emg_right_time = hdf['myo-right']['emg']['time_s'][:]
    #print(len(emg_left_data))
    #print(len(emg_right_data))
    print(hdf['eye-tracking-video-world']['frame_timestamp']['time_str'][0])  ## USA QUESTO PER I FRAME RGB
    print(hdf['eye-tracking-video-world']['frame_timestamp']['time_str'][-1])
    print(hdf['eye-tracking-video-world']['frame_timestamp']['time_s'][-1] - hdf['eye-tracking-video-world']['frame_timestamp']['time_s'][0])

    '''
    activities = [[x.decode('utf-8') for x in row] for row in hdf['experiment-activities']['activities']['data']]
    
    datasetsx = hdf['myo-left']
    datasetdx = hdf['myo-right']

    print(len(datasetsx['emg']['data']))
    #data_sx = datasetsx[:]
    #data_dx = datasetdx[:]

    # Stampa i dati (o fai altre operazioni con i dati)
    print("---------------Data from 'myo-left':")
    print(datasetsx.keys())
    print(datasetsx['emg'].keys())
    print(datasetsx['emg']['data'][0])

#    print("Data from 'myo-right':")
#    print(datasetdx.keys())
    

    # Replace with your path to one of the subjects from Action-Net
    #emg_annotations = pd.read_pickle("../../aml22-ego-solutions/action-net/emg_annotations/S04_1.pkl")
    dataset_emgSX = datasetsx['emg']
    sample_no = 1
    signal = torch.from_numpy(dataset_emgSX['data'][:]).float()
    title = 'myo-left emg signal'
    print(dataset_emgSX['time_str'][137552:137560])
    #compute_spectrogram(signal, title)
    print(f"FIRST DATA -----> data: {dataset_emgSX['data'][137552]} ,time: {dataset_emgSX['time_str'][137552]}, time_s_original: {dataset_emgSX['time_s_original'][137552]}, time_s: {dataset_emgSX['time_s'][137552]}")
    dataLabel = hdf['experiment-activities']['activities']
    print("---------------Data from 'experiment-activities':")
    print(dataLabel.keys())
    print(dataLabel['data'].shape)
    print(dataLabel['data'][0:2][:])
    sett = set()
    for (l,s,v,n) in dataLabel['data']:
        sett.add(n)
    
    for n in sett:
        print(n)
    
    print("---------------time_str-----------------:")
    print(dataLabel['time_str'][0:2])
    print(dataLabel['time_s'][1] - dataLabel['time_s'][0])
    print("---------------Data from 'experiment-notes':")
    dataLabel = hdf['experiment-notes']
    print(dataLabel.keys())
    print(dataLabel['notes']['data'][:,:])
    

    
    datasetsx = hdf['myo-left']['emg']
    print(datasetsx.keys())
    left_time_set = list(np.round(hdf['myo-left']['emg']['time_s'], 2))
    left_idx_set = set()
    for (i,el) in enumerate(list(np.round(hdf['myo-right']['emg']['time_s'], 2))):
        if el in left_time_set:
            left_index = left_time_set.index(el)
            left_idx_set.add(left_index)
            #print(f"Match! i: {i}, el: {el}, left_index: {left_index}")
        if i%3000 == 0 and i == 0:
            print(f"i: {i}, el: {el}")
            break
    for e in left_idx_set:
        print(f"left idx: {e}")
    print(f"-----------\n{len(left_idx_set)}\n------------")
    print(hdf['myo-right']['emg']['time_str'][-1])
    print(len(hdf['eye-tracking-video-eye']['frame_timestamp']['time_str']))
    print(hdf['eye-tracking-video-eye']['frame_timestamp']['time_str'][0])
    print(hdf['eye-tracking-video-eye']['frame_timestamp']['time_str'][-1])
    print(len(hdf['eye-tracking-video-world']['frame_timestamp']['time_str']))
    print(hdf['eye-tracking-video-world']['frame_timestamp']['time_str'][0])  ## USA QUESTO PER I FRAME RGB
    print(hdf['eye-tracking-video-world']['frame_timestamp']['time_str'][-1])
    print(hdf['eye-tracking-video-world']['frame_timestamp']['time_s'][108768] - hdf['eye-tracking-video-world']['frame_timestamp']['time_s'][0])
# Crea DataFrame per i dati della mano sinistra e destra
# Aggiungi i canali come colonne separate
df_left = pd.DataFrame(emg_left_data, columns=[f'emg_left_ch{i+1}' for i in range(emg_left_data.shape[1])])
df_left['time_s'] = emg_left_time

df_right = pd.DataFrame(emg_right_data, columns=[f'emg_right_ch{i+1}' for i in range(emg_right_data.shape[1])])
df_right['time_s'] = emg_right_time

# Unisci i DataFrame sui timestamp
merged_df = pd.merge_asof(df_left.sort_values('time_s'), df_right.sort_values('time_s'), on='time_s', direction='nearest')'''


FileNotFoundError: [Errno 2] Unable to synchronously open file (unable to open file: name = 'D:/ActionNetDataset/2022-06-07_18-11-37_streamLog_actionNet-wearables_S04_1.hdf5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

## Extract EMG

In [7]:


# Define the source directory containing the .hdf5 files and the destination directory for the .pkl files
source_dir = 'D:/ActionNetDataset/'
dest_dir = 'D:/ActionNetDataset/saved_emg/'

# List all .hdf5 files in the source directory
hdf5_files = [f for f in os.listdir(source_dir) if f.endswith('.hdf5')]

# Loop through each .hdf5 file and run the command
for hdf5_file in hdf5_files:
    source_filepath = os.path.join(source_dir, hdf5_file)
    name_path = hdf5_file.replace('.hdf5', '.pkl')
    dest = name_path.split('_')[-2] + '_' + name_path.split('_')[-1]
    dest_filepath = os.path.join(dest_dir, dest)

    
    command = [
        'python', 'extract_emg.py',
        f'source_filepath={source_filepath}',
        f'dest_filepath={dest_filepath}',
    ]
    
    result = subprocess.run(command, capture_output=True, text=True)
    
    # Print the output and error
    print('Output:', result.stdout)
    print('Error:', result.stderr)
    
    # Check the return code
    if result.returncode != 0:
        print(f'Command failed with return code {result.returncode}')
    else:
        print(f'Successfully processed {hdf5_file}')
    print(f'file = {dest}')
    print()

Output: 
Error: 
Successfully processed 2022-06-14_16-38-43_streamLog_actionNet-wearables_S04_1.hdf5
file = S04_1.pkl

Output: 
Error: 
Successfully processed 2022-06-13_18-14-59_streamLog_actionNet-wearables_S01_1.hdf5
file = S01_1.pkl

Output: 
Error: 
Successfully processed 2022-06-13_22-35-11_streamLog_actionNet-wearables_S02_3.hdf5
file = S02_3.pkl

Output: 
Error: 
Successfully processed 2022-06-13_23-22-44_streamLog_actionNet-wearables_S02_4.hdf5
file = S02_4.pkl

Output: 
Error: 
Successfully processed 2022-06-14_13-12-07_streamLog_actionNet-wearables_S03_1.hdf5
file = S03_1.pkl

Output: 
Error: 
Successfully processed 2022-06-14_13-52-57_streamLog_actionNet-wearables_S03_2.hdf5
file = S03_2.pkl

Output: 
Error: 
Successfully processed 2022-06-14_20-46-12_streamLog_actionNet-wearables_S05_2.hdf5
file = S05_2.pkl

Output: 
Error: 
Successfully processed 2022-07-12_14-31-04_streamLog_actionNet-wearables_S06_1.hdf5
file = S06_1.pkl

Output: 
Error: 
Successfully processed 2022-07-

In [4]:
from utils.actionNet_emg_record import ActionNetVideoEmgRecord


#try read emg

file_path = "D:/ActionNetDataset/saved_emg/S03_1.pkl"
df = pd.read_pickle(file_path)
'''
# Display the DataFrame
#print(f"durations = {df['end_time'] - df['start_time']}")
df.iloc[0]
el = ActionNetVideoEmgRecord(df.iloc[0].description,df.iloc[0].description_class,df.iloc[0].emg_data[0])
print(el.duration)
print(el.description)
print(el.label)
print(el.get_emg(1))
index = sorted(np.array(s04['index']) - 1)
print(index)
df_idx = df.iloc[index]
video_list = []
for i,(_,action) in enumerate(df_idx.iterrows()):
    if i < 10:
        descr = action['description']
        label = action['description_class']
        start = action['start_time']
        end = action['end_time']
        #print(action['description'])
        #print(action['description_class'])
        #print(action['start_time'])
        #print(action['end_time'])
        for el in action['emg_data']:
            sample = ActionNetVideoEmgRecord(descr,label, el)
            video_list.append(sample)
        
print(video_list[0])
    '''
df.emg_data[0][0][0]


{'time_s': 1655227278.399531,
 'left_emg': [-0.9529871704833096,
  -0.9274841832678671,
  -0.8948020986880154,
  -0.9402937530245367,
  -0.9406966260114197,
  -0.9354795261702438,
  -0.918537193947152,
  -0.9432085981480278],
 'right_emg': [-0.9018822612033881,
  -0.8795296668685978,
  -0.556958018102873,
  -0.5504120951277909,
  -0.6194903485221537,
  -0.7605260368916055,
  -0.8749520737659409,
  -0.9176174229005704]}

In [8]:
from utils.loaders import ActionNetEmgDataset

dataset = ActionNetEmgDataset('train', 10, 5, True, './action-net', 'D:/ActionNetDataset/saved_emg', 2)
(el,l) = dataset.__getitem__(0)
e = el.reshape(1, 5, 10, -1)  #(num_batch, num_clips, num_frames, num_features)
print(e.shape)  # Controlla la forma dell'array
len(dataset)
#10,24

(1, 5, 10, 16)


1795

## Extract RGB

In [ ]:
import os
from PIL import Image

def check_images(frames_folder: str):
    """
    Check if all images in the folder are valid and not corrupted.
    """
    valid_images = 0
    corrupted_images = 0

    for filename in os.listdir(frames_folder):
        if filename.endswith(".jpg"):
            filepath = os.path.join(frames_folder, filename)
            try:
                with Image.open(filepath) as img:
                    img.verify()  # Verifica che il file immagine sia valido
                valid_images += 1
            except (IOError, SyntaxError) as e:
                print(f"Corrupted image: {filepath}")
                corrupted_images += 1
        if valid_images % 100 == 0:
            print(f"Total valid images: {valid_images}")
            print(f"Total corrupted images: {corrupted_images}")

    print(f"Total valid images: {valid_images}")
    print(f"Total corrupted images: {corrupted_images}")


check_images('D:/ActionNetDataset/saved_RGB/camera1')

# PROVA

In [ ]:
from utils.loaders import FeaturesDataset
from torch.utils.data import DataLoader

train_dataset = FeaturesDataset("./saved_features/saved_feat_I3D_10_dense_D1",'test')
#test_dataset = PACSDataset(domain='sketch', transform=dataset_transform)
print(f"Lunghezza = {len(train_dataset)}\n\n")
# Define the DataLoaders
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, num_workers=4, drop_last=True)

for i_val,(x, y) in (enumerate(train_loader)):
    print(f"{i_val}) x({x.shape}) = {x}  \ny={y}")